In [1]:
%cd /data/codes/sb-apa/

from src.brain import get_brain_class
from hyperpyyaml import load_hyperpyyaml
import speechbrain as sb
import torch
import json
import sys
import os

from utils.arpa import arpa_to_ipa
import pandas as pd
import re


/data/codes/sb-apa


/home/tuyendv/miniconda3/envs/e2er/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_state_dict(hparams):
    wav2vec2_ckpt_path = f'{ckpt_path}/wav2vec2.ckpt'
    model_ckpt_path = f'{ckpt_path}/model.ckpt'

    wav2vec2_state_dict = torch.load(wav2vec2_ckpt_path)
    model_state_dict = torch.load(model_ckpt_path)

    hparams["wav2vec2"].load_state_dict(wav2vec2_state_dict)
    hparams["model"].load_state_dict(model_state_dict)

    return hparams

def init_model(hparams):
    brain_class = get_brain_class(hparams)

    model = brain_class(
            modules=hparams["modules"],
            hparams=hparams,
            run_opts=run_opts,
            checkpointer=hparams["checkpointer"],
        )

    hparams = load_state_dict(hparams)
    
    for key, value in hparams["modules"].items():
        value.eval()
    
    return model, hparams


In [3]:
DATA_DIR = "data"
APR_DATA_FOLDER = f'{DATA_DIR}/apr/'

RESULTS_FOLDER = f'{DATA_DIR}/results/'
EXP_METADATA_FILE = f'{RESULTS_FOLDER}/exp_metadata.csv'
APR_RESULTS_FILE = f'{RESULTS_FOLDER}/results_scoring.csv'
EPOCH_RESULTS_DIR = f'{RESULTS_FOLDER}/epoch_results'
PARAMS_DIR= f'{RESULTS_FOLDER}/params'


MODEL_TYPE = "w2v2"
SCORING_TYPE=""

APR_MODEL_DIR = f"pretrained/apr"
PRETRAINED_MODEL_DIR = f"pretrained/apr"
SCORING_HPARAM_FILE = f"hparams/apr.yml"

argv = [
    SCORING_HPARAM_FILE,
    "--data_folder", APR_DATA_FOLDER,
    "--exp_folder", APR_MODEL_DIR,
    "--batch_size", "4",
    "--exp_metadata_file", EXP_METADATA_FILE,
    "--results_file", APR_RESULTS_FILE,
    "--epoch_results_dir", EPOCH_RESULTS_DIR,
    "--params_dir", PARAMS_DIR
    ]

In [4]:
hparams_file, run_opts, overrides = sb.parse_arguments(argv)
with open(hparams_file) as fin:
    hparams = load_hyperpyyaml(fin, overrides)

lexicon_path = "resources/lexicon"
ckpt_path = "results/apr/save/best"
label_encoder_path = "results/apr/save/label_encoder.txt"

hparams["ckpt_path"] = ckpt_path
hparams["label_encoder_path"] = label_encoder_path
label_encoder_path = hparams["label_encoder_path"]

prep_model, hparams = init_model(hparams)
label_encoder = sb.dataio.encoder.CTCTextEncoder.from_saved(label_encoder_path)


Some weights of the model checkpoint at microsoft/wavlm-large were not used when initializing WavLMModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMModel were not initialized from the model checkpoint at microsoft/wavlm-large and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [5]:
from src.data import apr_dataio_prep

train_data, valid_data, test_data, label_encoder = apr_dataio_prep(hparams, label_encoder)

In [6]:
sample = valid_data[0]
sample

{'id': 'MBWM0_SI1934.WAV',
 'sig': tensor([3.0518e-04, 3.0518e-05, 9.1553e-05,  ..., 3.0518e-05, 3.9673e-04,
         1.8311e-04]),
 'phn_encoded': tensor([43,  8, 10, 16, 17, 23, 42, 16, 23,  8, 21, 23, 43]),
 'phn_encoded_eos': tensor([43,  8, 10, 16, 17, 23, 42, 16, 23,  8, 21, 23, 43,  1]),
 'phn_encoded_bos': tensor([ 0, 43,  8, 10, 16, 17, 23, 42, 16, 23,  8, 21, 23, 43])}

In [20]:
phns = label_encoder.decode_ndim(sample["phn_encoded"])
phns

['sil', 'w', 'ey', 't', 'ah', 'l', 'ih', 't', 'l', 'w', 'ay', 'l', 'sil']

In [12]:
wavs = sample["sig"].unsqueeze(0).cuda()
wav_lens = torch.tensor([wavs.shape[1]]).cuda()
phn_encoded = sample["phn_encoded"].unsqueeze(0).cuda()
phns_eos = sample["phn_encoded_eos"].unsqueeze(0).cuda()
phns_bos = sample["phn_encoded_bos"].unsqueeze(0).cuda()

In [14]:
p_ctc, p_seq, wav_lens = prep_model.infer(wavs, wav_lens, phns_bos)

In [22]:
p_seq.shape

torch.Size([1, 14, 46])

In [16]:
p_ctc.shape

torch.Size([1, 54, 46])